# Clone an Example from S3

## Overview
In this lesson we will clone a small example TEEHR Evaluation from S3 and run through some simple example metrics calculations, demonstrating filter, grouping and chaining of query methods.

### Create a new Evaluation
First we will import TEEHR along with some other required libraries for this example.  Then we create a new instance of the Evaluation that points to a directory where the evaluation data will be stored.

In [ ]:
import teehr
from pathlib import Path
import shutil

# Tell Bokeh to output plots in the notebook
from bokeh.io import output_notebook
output_notebook()

In [ ]:
# Define the directory where the Evaluation will be created
test_eval_dir = Path(Path().home(), "temp", "04_setup_real_example")
shutil.rmtree(test_eval_dir, ignore_errors=True)

# Create an Evaluation object and create the directory
ev = teehr.Evaluation(dir_path=test_eval_dir, create_dir=True)

### Clone Evaluation Data form S3
As mentioned above, for this exercise we will be cloning a complete Evaluation dataset from the TEEHR S3 bucket.  First we will list the available Evaluations and then we will clone the `e0_2_location_example` evaluation which is  a small example Evaluation that only contains 2 gages.

In [ ]:
# List the evaluations in the S3 bucket
ev.list_s3_evaluations()

In [ ]:
# Clone the e0_2_location_example evaluation from the S3 bucket
ev.clone_from_s3("e0_2_location_example")

Now that we have cloned all the data for this evaluation from the TEEHR S3 bucket, lets query the `locations` table as a GeoPandas GeoDataFrame and then plot the gages on a map using the TEEHR plotting.

In [ ]:
locations_gdf = ev.locations.to_geopandas()
locations_gdf.teehr.location_map()

Lets also query the `primary_timeseries` and plot the timeseries data using the `df.teehr.timeseries_plot()` method.

In [ ]:
pt_df = ev. primary_timeseries.to_pandas()
pt_df.head()

In [ ]:
pt_df.teehr.timeseries_plot()

And the `location_crosswalks` table.

In [ ]:
lc_df = ev.location_crosswalks.to_pandas()
lc_df.head()

And the `secondary_timeseries` table.

In [ ]:
st_df = ev.secondary_timeseries.to_pandas()
st_df.head()

In [ ]:
st_df.teehr.timeseries_plot()

And lastly, the `joined_timeseries` table.

In [ ]:
jt_df = ev.joined_timeseries.to_pandas()
jt_df.head()

### Metrics
Now that we have confirmed that we have all the data tables and the `joined_timeseries` table, we can move on to analyzing the data.  The user is encouraged to check out the documentation pages relating to filtering and grouping in the context of generating metrics.  The short explanation is that `filters` can be used to select what values are used when calculating metrics, while the `group_by` determines how the values are grouped into populations before calculating metrics.

The most basic way to evaluate simulation performance is to `group_by` `configuration_name` and `primary_location_id`, and generate some basic metrics.  In this case it will be Nash-Sutcliffe Efficiency, Kling-Gupta Efficiency and Relative Bias, calculated at each location for each configuration.  As we saw there are 2 locations and 1 configuration, so the total number of rows that are output is just 2.  If there were more `locations` or more `configurations`, there would be more rows in the output for this query.  TEEHR contains many more metrics that can be calculated by simply including them in the list of `include_metrics`, and there are also many other ways to look at performance besides the basic metrics.

In [ ]:
ev.metrics.query(
    group_by=["configuration_name", "primary_location_id"],
    include_metrics=[
        teehr.Metrics.NashSutcliffeEfficiency(),
        teehr.Metrics.KlingGuptaEfficiency(),
        teehr.Metrics.RelativeBias()
    ]
).to_pandas()

Now to demonstrate how filters work, if we add a filter to only select values where the `primary_location_id` is `usgs-14138800`.  Accordingly, it will only include rows from the `join_timeseries` table where `primary_location_id` is `usgs-14138800` in the metrics calculations, and since we are grouping by `primary_location_id`, that means we can expect one row in the output.  And that is what we see below.

In [ ]:
(
    ev.metrics
    .query(
        group_by=["configuration_name", "primary_location_id"],
        filters=[
            {
                "column": "primary_location_id",
                "operator": "=",
                "value": "usgs-14138800"
            }],
        include_metrics=[
            teehr.Metrics.NashSutcliffeEfficiency(),
            teehr.Metrics.KlingGuptaEfficiency(),
            teehr.Metrics.RelativeBias()
        ]
    )
    .to_pandas()
)

As another example, because the `joined_timeseries` table contains a `year` column which was added as a user defined field, we can also group by `year`.  In this case we will get the metrics calculated for each `configuration_name`, `primary_location_id`, and `year`.

In [ ]:
(
    ev.metrics
    .query(
        group_by=["configuration_name", "primary_location_id", "year"],
        filters=[
            {
                "column": "primary_location_id",
                "operator": "=",
                "value": "usgs-14138800"
            }],
        include_metrics=[
            teehr.Metrics.NashSutcliffeEfficiency(),
            teehr.Metrics.KlingGuptaEfficiency(),
            teehr.Metrics.RelativeBias()
        ]
    )
    .to_pandas()
)

There are many ways that TEEHR can be used to "slice and dice" the data in the TEEHR dataset.  One last example here before wrapping up this lesson.  Lets say we wanted the "annual peak relative bias", so that is the relative bias of the annual peak values.  Well, TEEHR can do this too by chaining the query methods together and overriding the `input_field_names` and the `output_field_name` as shown below.  We will do this step by step to understand it. First run the following query where the second `query` is commented out then in the next cell run it with the second `query` uncommented.  As you can see first we calculate the peak primary value (`max_primary_value`) and peak secondary value (`max_secondary_value`) for each year, then we calculate the relative bias across the yearly peaks (`annual_max_relative_bias`).

In [ ]:
(
    ev.metrics
    .query(
        group_by=["configuration_name", "primary_location_id", "year"],
        filters=[
            {
                "column": "primary_location_id",
                "operator": "=",
                "value": "usgs-14138800"
            }],
        include_metrics=[
            teehr.Metrics.Maximum(
                input_field_names=["primary_value"],
                output_field_name="max_primary_value"
            ),
            teehr.Metrics.Maximum(
                input_field_names=["secondary_value"],
                output_field_name="max_secondary_value"
            )
        ]
    )
    # .query(
    #     group_by=["configuration_name", "primary_location_id"],
    #     include_metrics=[
    #         teehr.Metrics.RelativeBias(
    #             input_field_names=["max_primary_value", "max_secondary_value"],
    #             output_field_name="monthly_max_relative_bias"
    #         )
    #     ]
    # )
    .to_pandas()
)

In [ ]:
(
    ev.metrics
    .query(
        group_by=["configuration_name", "primary_location_id", "year"],
        filters=[
            {
                "column": "primary_location_id",
                "operator": "=",
                "value": "usgs-14138800"
            }],
        include_metrics=[
            teehr.Metrics.Maximum(
                input_field_names=["primary_value"],
                output_field_name="max_primary_value"
            ),
            teehr.Metrics.Maximum(
                input_field_names=["secondary_value"],
                output_field_name="max_secondary_value"
            )
        ]
    )
    .query(
        group_by=["configuration_name", "primary_location_id"],
        include_metrics=[
            teehr.Metrics.RelativeBias(
                input_field_names=["max_primary_value", "max_secondary_value"],
                output_field_name="annual_max_relative_bias"
            )
        ]
    )
    .to_pandas()
)

In [ ]:
ev.spark.stop()